In [ ]:
from timeseries_fastai.imports import *
from timeseries_fastai.data import *
from timeseries_fastai.core import *
from timeseries_fastai.models import *
from timeseries_fastai.tabular import *

from ucr import *

In [ ]:
PATH.ls()

(#135) [Path('/home/tcapelle/.fastai/data/Univariate2018_arff/GestureMidAirD2'),Path('/home/tcapelle/.fastai/data/Univariate2018_arff/Adiac'),Path('/home/tcapelle/.fastai/data/Univariate2018_arff/CinCECGTorso'),Path('/home/tcapelle/.fastai/data/Univariate2018_arff/LargeKitchenAppliances'),Path('/home/tcapelle/.fastai/data/Univariate2018_arff/MixedShapesRegularTrain'),Path('/home/tcapelle/.fastai/data/Univariate2018_arff/ShakeGestureWiimoteZ'),Path('/home/tcapelle/.fastai/data/Univariate2018_arff/SmallKitchenAppliances'),Path('/home/tcapelle/.fastai/data/Univariate2018_arff/CricketY'),Path('/home/tcapelle/.fastai/data/Univariate2018_arff/ToeSegmentation2'),Path('/home/tcapelle/.fastai/data/Univariate2018_arff/EOGVerticalSignal')...]

In [ ]:
NON_NAN_TASKS[0:5]

['ACSF1', 'Adiac', 'ArrowHead', 'BME', 'Beef']

In [ ]:
def max_bs(N):
    N = N//6
    k=1
    while (N//2**k)>1: k+=1
    return min(2**k, 32)

In [ ]:
def get_dls(path, task, bs=None, workers=None):
    df_train, df_test = load_df_ucr(path, task)
    bs = ifnone(bs, max_bs(len(df_train)))
    x_cols = df_train.columns[0:-1].to_list()
    return TSDataLoaders.from_dfs(df_train, df_test, x_cols=x_cols, label_col='target', 
                                  bs=bs, num_workers=workers)

In [ ]:
dls = get_dls(PATH, 'Adiac')

Loading files from: /home/tcapelle/.fastai/data/Univariate2018_arff/Adiac


In [ ]:
def get_dls2(path, task, bs=None, workers=None):
    df_train, df_test = load_df_ucr(path, task)
    bs = ifnone(bs, max_bs(len(df_train)))
    x_cols = df_train.columns[0:-1].to_list()
    
    df_main = stack_train_valid(df_train, df_test)
    splits=[range_of(df_train), list(range(len(df_train), len(df_main)))]
    to = TSPandas(df_main, [Normalize], x_names=x_cols, y_names='target', splits=splits)
    
    return to.dataloaders(bs, 2*bs)

In [ ]:
dls = get_dls2(PATH, 'Adiac')

Loading files from: /home/tcapelle/.fastai/data/Univariate2018_arff/Adiac


In [ ]:
dls.c

37

In [ ]:
x,y = dls.train.one_batch()

In [ ]:
x.shape

torch.Size([32, 1, 176])

In [ ]:
def get_model(dls, arch):
    num_classes = dls.c
    arch = arch.lower()
    if arch=='resnet':     model = create_resnet(1, num_classes, conv_sizes=[64, 128, 256])
    elif arch=='fcn':      model = create_fcn(1, num_classes, ks=9, conv_sizes=[128, 256, 128])
    elif arch=='mlp':      model = create_mlp(dls.train.one_batch()[0].shape[-1], num_classes)
    elif arch=='inception':model = create_inception(1, num_classes)
    else: 
        print('Please chosse a model in [resnet, FCN, MLP, inception]')
        return None
    return model

In [ ]:
LabelSmoothingCrossEntropy??

Init signature: LabelSmoothingCrossEntropy(eps: float = 0.1, reduction='mean')
Docstring:      Same as `nn.Module`, but no need for subclasses to call `super().__init__`
Source:        
class LabelSmoothingCrossEntropy(Module):
    y_int = True
    def __init__(self, eps:float=0.1, reduction='mean'): self.eps,self.reduction = eps,reduction

    def forward(self, output, target):
        c = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        if self.reduction=='sum': loss = -log_preds.sum()
        else:
            loss = -log_preds.sum(dim=-1) #We divide by that size at the return line so sum and not mean
            if self.reduction=='mean':  loss = loss.mean()
        return loss*self.eps/c + (1-self.eps) * F.nll_loss(log_preds, target.long(), reduction=self.reduction)

    def activation(self, out): return F.softmax(out, dim=-1)
    def decodes(self, out):    return out.argmax(dim=-1)
File:           ~/Apps/fastai/fastai/losses.py
Type:           PrePostIn

In [ ]:
def train_task(path, task='Adiac', arch='resnet', epochs=40, lr=5e-4):
    "trains arch over task with params"
    dls = get_dls2(path, task)
    model = get_model(dls, arch)
    cbs =  MixUp(0.2)
    learn = Learner(dls, model=get_model(dls, arch), opt_func=Adam, \
            metrics=[accuracy])
    learn.fit_one_cycle(epochs, lr, cbs=cbs)                          
    return learn.to_fp16()

In [ ]:
learn = train_task(PATH, 'Adiac', arch='mlp', epochs=4)

Loading files from: /home/tcapelle/.fastai/data/Univariate2018_arff/Adiac


epoch,train_loss,valid_loss,accuracy,time
0,3.609463,3.422132,0.102302,00:00
1,3.206733,2.765023,0.373402,00:00
2,2.919268,2.339994,0.383632,00:00
3,2.743875,2.223814,0.391304,00:00


In [ ]:
def compute_metrics(learn):
    "compute oguiza Metrics on UCR"
    results = np.array(learn.recorder.values)
    acc_ = results[-1,-1]
    accmax_ = results[:, -1].max()
    loss_ = results[-1,0]
    val_loss_ = results[-1,1]
    return acc_, accmax_, loss_, val_loss_ 

In [ ]:
learn = train_task(PATH, NON_NAN_TASKS[0], epochs=4)

Loading files from: /home/tcapelle/.fastai/data/Univariate2018_arff/ACSF1


epoch,train_loss,valid_loss,accuracy,time
0,2.112109,1.962039,0.300000,00:00
1,1.808008,1.624210,0.630000,00:00
2,1.611130,1.334273,0.560000,00:00
3,1.459213,1.106631,0.720000,00:00


In [ ]:
def run_tasks(tasks, arch='resnet', lr=1e-3, epochs=1):
    results = [compute_metrics(train_task(PATH, task, arch, epochs, lr)) for task in tasks]
    return pd.DataFrame(data=results, columns=['acc', 'acc_max', 'train_loss', 'val_loss'], index=tasks)

In [ ]:
run_tasks(NON_NAN_TASKS)

Loading files from: /home/tcapelle/.fastai/data/Univariate2018_arff/ACSF1


epoch,train_loss,valid_loss,accuracy,time
0,1.928175,1.908550,0.320000,00:00


Loading files from: /home/tcapelle/.fastai/data/Univariate2018_arff/Adiac


epoch,train_loss,valid_loss,accuracy,time
0,3.271958,3.263362,0.143223,00:02


Loading files from: /home/tcapelle/.fastai/data/Univariate2018_arff/ArrowHead


...

Loading files from: /home/tcapelle/.fastai/data/Univariate2018_arff/Car


epoch,train_loss,valid_loss,accuracy,time
0,1.455936,1.438430,0.416667,00:00


Loading files from: /home/tcapelle/.fastai/data/Univariate2018_arff/Chinatown


epoch,train_loss,valid_loss,accuracy,time
0,0.894555,0.576823,0.666667,00:00


KeyboardInterrupt: 